In [3]:
import sklearn
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from tkinter import *
from tkinter import ttk
main_window = Tk()
main_window.geometry('700x400')
main_window.title("Plagiarism Checker")

Label(main_window,text = "Please enter the location to the files you want to check plagiasm:",font=('Courier New',10)).grid(row = 0, column=0)

#C:\Users\ankit\Desktop\Plagiarism Checker\Assignment
p = Entry(main_window,width = 60)
p.grid(row = 1, column = 0)

Label(main_window,text = "Enter the type of file:",font=('Courier New',10)).grid(row = 2, column = 0)

n =StringVar()
lchose=ttk.Combobox(main_window, width = 60, textvariable = n)
lchose['values']=('text','python','cpp','java')
lchose.grid(row=3,column=0)

global student_files
global student_notes 
global s_vectors
def c():
   
    path = p.get()
    if lchose.get() == "text":
        student_files = [doc for doc in os.listdir(path) if doc.endswith('.txt')]
    elif lchose.get() == "python":
        student_files = [doc for doc in os.listdir(path) if doc.endswith('.ipynb')]
    elif lchose.get() == "cpp":
        student_files = [doc for doc in os.listdir(path) if doc.endswith('.cpp')]
    student_notes = [open(_file, encoding='utf-8').read()
                 for _file in student_files]
    def vectorize(Text): return TfidfVectorizer().fit_transform(Text).toarray()
    def similarity(doc1, doc2): return cosine_similarity([doc1, doc2])
    vectors = vectorize(student_notes)
    s_vectors = list(zip(student_files, vectors))
    plagiarism_results = set()
    def check_plagiarism():
        for student_a, text_vector_a in s_vectors:
            new_vectors = s_vectors.copy()
            current_index = new_vectors.index((student_a, text_vector_a))
            del new_vectors[current_index]
            for student_b, text_vector_b in new_vectors:
                sim_score = similarity(text_vector_a, text_vector_b)[0][1]
                sim_score = round(sim_score, 4)
                student_pair = sorted((student_a, student_b))
                score = (student_pair[0], student_pair[1], sim_score)
                plagiarism_results.add(score)
        return plagiarism_results
    check_plagiarism()
    result=""
    for data in check_plagiarism():
        if data[2] > .70:
            result += data[0] + " and " + data[1] + " have plagiarism " + "with percentage " + str(data[2])+ ".\n" 
    main = Toplevel(main_window)
    main.geometry('700x400')
    main.title("Result")
    Label(main,text = "Here is the result after check:",font=('Courier New',10)).grid(row = 1, column=0)
    Label(main,text = result ,font=('Courier New',10), foreground="red").grid(row = 2, column =0)

    
button = Button(main_window,text = "Click to Check",font=('Courier New',10), command = c)
button.grid(row = 4, column = 0)

main_window.mainloop()